In [54]:
#Preparing directory in Local FileSystem 
%cd ~/notebook/work/
!mkdir BDCW
%cd ~/notebook/work/BDCW
# Getting file from UCI repository
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00438/Health-News-Tweets.zip
!unzip -o -q Health-News-Tweets.zip -d ~/notebook/work/BDCW
# Deleting Unused Folder
!rm -r __MACOSX
!rm  Health-News-Tweets.zip
# Setting it as home directory 
%cd ~/notebook/work/BDCW/Health-Tweets
#cause pyspark bug, probably strange characters > removing...
!rm goodhealth.txt
# Displaying files
!ls

/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb58-fd12fb10398921-2dd0a6f275af/notebook/work
mkdir: cannot create directory ‘BDCW’: File exists
/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb58-fd12fb10398921-2dd0a6f275af/notebook/work/BDCW
--2018-03-26 05:16:16--  https://archive.ics.uci.edu/ml/machine-learning-databases/00438/Health-News-Tweets.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3452966 (3.3M) [application/zip]
Saving to: ‘Health-News-Tweets.zip’

100%[======================================>] 3,452,966   5.48MB/s   in 0.6s   

2018-03-26 05:16:17 (5.48 MB/s) - ‘Health-News-Tweets.zip’ saved [3452966/3452966]

/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb58-fd12fb10398921-2dd0a6f275af/notebook/work/BDCW/Health-Tweets
bbchealth.txt	    foxnewshealth.txt	  msnhealthnews.txt  reuters_health.txt
cbchealt

In [1]:
# Load Spark context 
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [55]:
from pathlib import Path
import re

# Setting default working directory
%cd ~/notebook/work/BDCW
# Define function lockandload needed to run through 
def LockAndLoad(address):

    p = Path(address)
    dirs = list(p.iterdir())

    print(dirs)
    rddarray=[]
    filenames=[]

    for i in dirs:
        text = sc.textFile(str(i.absolute())).map(lambda a: (a.split('|')))
        name = sc.wholeTextFiles(str(i.absolute()))
        name = name.map(lambda x: x[0])
        filenames.append(name)
        text.take(10)
        #fn = fn.map(lambda a: (a.split('|')))
        rddarray.append(text)
    
    print('len(rddarray)',len(rddarray))
    print('len(filenames)',len(filenames))

    full_rdd = sc.parallelize(str(""))
        
    for i in range (0,15):
        p = str(filenames[i].take(1))
        nm = re.search('Health-Tweets(.*?).txt', p).group(1)
        rdd = rddarray[i].map(lambda x: (str(nm),x[2],x[1]))
        full_rdd = full_rdd.union(rdd)

    return full_rdd


full_rdd = LockAndLoad('Health-Tweets')

/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb58-fd12fb10398921-2dd0a6f275af/notebook/work/BDCW
[PosixPath('Health-Tweets/gdnhealthcare.txt'), PosixPath('Health-Tweets/NBChealth.txt'), PosixPath('Health-Tweets/usnewshealth.txt'), PosixPath('Health-Tweets/KaiserHealthNews.txt'), PosixPath('Health-Tweets/reuters_health.txt'), PosixPath('Health-Tweets/bbchealth.txt'), PosixPath('Health-Tweets/msnhealthnews.txt'), PosixPath('Health-Tweets/foxnewshealth.txt'), PosixPath('Health-Tweets/cbchealth.txt'), PosixPath('Health-Tweets/cnnhealth.txt'), PosixPath('Health-Tweets/latimeshealth.txt'), PosixPath('Health-Tweets/nytimeshealth.txt'), PosixPath('Health-Tweets/nprhealth.txt'), PosixPath('Health-Tweets/wsjhealth.txt'), PosixPath('Health-Tweets/everydayhealth.txt')]
len(rddarray) 15
len(filenames) 15


In [58]:
from pyspark.sql import functions as F
from pyspark.sql.functions import concat, col, lit, desc
import matplotlib.pyplot as plt
#Count data by time:

def countby(rdd, var):
    # delete any previous pending plots, if any...
    plt.clf()
    # transform rdd to dataframe object
    stg_ssn = full_rdd.map(lambda p: Row(service=p[0], text=p[1], date=p[2]))
    stg = sqlContext.createDataFrame(full_rdd, ['service','text','date'])
    # Setting time to legible format:
    stg = stg.withColumn('date', concat(stg.date.substr(5,3)))
    # Count by var, it can be service or date (months and years)
    counts = stg.groupBy([var]).count().alias('counts').sort(desc("count"))
    # reduced df is transformed into pandas format
    df = counts.toPandas()
    # plotting...
    df.plot(kind='bar',x=var,y='count',colormap='winter_r')
    plt.show()

countby(full_rdd, 'date')
countby(full_rdd, 'service')

In [73]:
stg_ssn = full_rdd.map(lambda p: Row(service=p[0], text=p[1], date=p[2]))
stg = sqlContext.createDataFrame(full_rdd, ['service','text','date'])
stg.show()

+--------------+--------------------+--------------------+
|       service|                text|                date|
+--------------+--------------------+--------------------+
|/gdnhealthcare|Are you a member ...|Wed Apr 08 19:48:...|
|/gdnhealthcare|What is palliativ...|Wed Apr 08 18:47:...|
|/gdnhealthcare|Most viewed this ...|Wed Apr 08 17:49:...|
|/gdnhealthcare|How can technolog...|Wed Apr 08 17:41:...|
|/gdnhealthcare|In case you misse...|Wed Apr 08 16:39:...|
|/gdnhealthcare|"India is a stark...|Wed Apr 08 13:45:...|
|/gdnhealthcare|Popular on the ne...|Wed Apr 08 12:43:...|
|/gdnhealthcare|Are there any top...|Wed Apr 08 11:52:...|
|/gdnhealthcare|Lots of debate go...|Wed Apr 08 10:40:...|
|/gdnhealthcare|Our weekly newsle...|Wed Apr 08 10:06:...|
|/gdnhealthcare|New today: I've s...|Wed Apr 08 08:39:...|
|/gdnhealthcare|More than 140 top...|Wed Apr 08 08:32:...|
|/gdnhealthcare|Don't miss: Techn...|Tue Apr 07 19:53:...|
|/gdnhealthcare|Most read today: ...|Tue Apr 07 18:50:..

In [57]:
import nltk
import re
# define a function for tokenizing and stemming words
def stemAndtokenize(text_final): 
    nltk.download('punkt')
    p = nltk.PorterStemmer()
    # stemming words 
    tokens = p.stem(text_final)
    # tokenizing words:
    tokens = nltk.word_tokenize(text_final)
    tokens2 = [re.sub('[^A-Za-z0-9]+', '',x) for x in tokens]
    #tokens3 = tokens2.map(lambda x: (x[0][1:], [word for word in x[1] if len(word)>0]))
    
    return tokens2

def refiner(text):
    tokens = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '',text)
    from nltk.corpus import stopwords
    nltk.data.path.append("/nltk_data")
    stop = stopwords.words('english')
    avoid = ['What', 'I', 'what', 'A', 'US', 'How', 'may', 'says', 'Most', 'most', 'Health', 'health', 'healthcare', 'The', 'HealthTalk', 
                    'healthtalk', 'To', 'In', 'to', 'in', 'us', 'New', 'Well', 'May', 'Why', 'For', 'Is', 'new', 'E', 'EverydayHealth', 'RT', 'NHS']
    stop.extend(avoid)
    pattern = re.compile(r'\b(' + r'|'.join(stop) + r')\b\s*')
    text_final = pattern.sub('', tokens)
    
    print(text_final)
    
    return text_final

In [6]:
from operator import add

# we want to visualize word by counting them by months,years to see which word has been used the most for a certain month,year.

# Create key-values pair with text on x[1] and dates on x[2]
rdd1 = full_rdd.map(lambda x: (x[2], (x[1])))
# take values
rdd2 = rdd1.values()
# call text refiner, remove URLs and stopwords 
rdd3 = rdd2.map(refiner)
# call tokenizer, tokenize and keep only words, removing hashtags, punctuation, lowerings and removing plurals....
rdd4 = rdd3.map(stemAndtokenize)
# Zipp keys and values together... 
rdd6 = rdd1.keys().zip(rdd4)
# Setting date to a readable format and delete whitespaces
rdd7 = rdd6.map(lambda x: (x[0][4:7]+(x[0][-4:]), [s for s in x[1] if len(s)>0]))
# Erease eventual white spaces...
rdd_final = rdd7.map(lambda x: (x[0][0:], [word for word in x[1] if len(word)>0]))

def countbyMonths(rdd):
    # Collect into array all the months/years
    months = rdd.map(lambda x: x[0]).distinct().collect()
    # Inizialize an Empty RDD
    outputs = sc.parallelize(str(""))
        # Iterate through all the months, this could take some times... go have a tea...
    for i in months:
        # Take all the observations with that particular months and yeara(example: all the text with "oct2014")
        t1 = rdd.filter(lambda x: x[0]==str(i))
        # flatMap to produce touple with key and value (word,1) of the sorted RDD
        t2 = t1.flatMap(lambda x: (x[1])).map(lambda x: (x,1))
        # count by key, to obtain the number of words for that particular month/years combination
        t3 = t2.reduceByKey(add)
        # Order by count
        t4 = t3.sortBy(lambda x: -x[1])
        # attach months/years key to word,count touple.
        t5 = t4.map(lambda x: (str(i) ,x[0], x[1]))
        # insert the result in the empty rdd
        outputs = outputs.union(t5)
        
    return outputs
          
# Uncomment to re-run the long code.
rdd_df = countbyMonths(rdd_final)
# Create schema for the dataframe
df = rdd_df.map(lambda x: Row(month=x[0], word=x[1], count=x[2]))
# Create dataframe using the schema.
df = sqlContext.createDataFrame(rdd_df, ['month','word','count'])
# Sort descending order
df = df.sort(desc("count"))
# Print dataframe...
df.show()


+-------+-------+-----+
|  month|   word|count|
+-------+-------+-----+
|Oct2014|  Ebola| 1678|
|Aug2014|  Ebola|  602|
|Sep2014|  Ebola|  481|
|Nov2014|  Ebola|  425|
|Dec2014|  Ebola|  304|
|Oct2014|     US|  283|
|Jan2015|    amp|  193|
|Jan2015|  Ebola|  169|
|Oct2014| Dallas|  161|
|Sep2014|     US|  145|
|Oct2014|patient|  144|
|Mar2015|  Ebola|  138|
|Oct2014| Africa|  126|
|Jul2014|  Ebola|  123|
|Jan2013|  foods|  123|
|Feb2013|  foods|  120|
|Oct2014|  Texas|  117|
|Oct2014|    CDC|  113|
|Aug2014|   drug|  112|
|Aug2014|     US|  109|
+-------+-------+-----+
only showing top 20 rows



In [8]:
# We want to predict the author of the tweet using a multiclass logistic regression, we need first to work on the RDD, deleting stopwords, tokenizing,
# and refining.

# Initialize dataset:
# You can pass inside the initializer an RDD and it will process it by tokenizing, stopword removing, stemming and returning a processed RDD
def initialize(rdd):
# Define the RDD to process, define a tuple of keys and values, where keys are x[1] and they will be our label, and word are the second part of our tuple.  
    rdd = rdd.map(lambda x: (x[0], (x[1])))
    # separate the values from the keys, to work on just on the text 
    values = rdd.values()
    # call the refiner function on the rdd
    rdd_refined = values.map(refiner)
    # call the stemmer and tokenizer function on rdd
    rdd_tokenized = rdd_refined.map(stemAndtokenize)
    # re-zip keys and values together.
    zipped_rdd = rdd.keys().zip(rdd_tokenized)
    # remove empty spaces from RDD due to the refiner and tokenizer actions.
    model_rdd = zipped_rdd.map(lambda x: (x[0][1:], [word for word in x[1] if len(word)>0]))
    
    return model_rdd
    
    

# This function is meant to convert the RDD in a dataframe processable by pyspark.ml library --- we want to predict the author of the tweet, pyspark.ml 
# need a dataframe object so we need to convert before proceed to the machine learning part.
def to_indexed_dataframe(rdd):
    # define column name for the dataframe
    columns = ['accounts', 'text']
    # transform the rdd into a df with the 
    df = rdd.toDF(columns)
    # transform the tweet author name into a numerical index processable for multiclass classification.
    indexer = StringIndexer(inputCol="accounts", outputCol="label")
    indexed_df = indexer.fit(df).transform(df)
    
    return indexed_df

In [52]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA

def makeLda(pippo):
    # create vector object
    cv = CountVectorizer(inputCol="text", outputCol="vectors")

    #map action. model fitted on the single workers
    lda_an=cv.fit(pippo)
    vocab= lda_an.vocabulary

    #reduce action, all the map fit actions are reduced/put togheter, and then showed
    #lda_an.transform(lda_df).show()
    lda_trans= lda_an.transform(lda_df)

    #specify parameters of LDA; k=2 number of topics. 
    lda = LDA(k=10, seed=1, optimizer="em", featuresCol='vectors')

    #run LDA on 
    model = lda.fit(lda_trans)
    return model, vocab    

In [74]:
full_rdd= LockAndLoad('Health-Tweets')
#start from rdd structure
lda_rdd= initialize(full_rdd)
#print(lda_rdd.take(3))
df= to_indexed_dataframe(lda_rdd)

#call function pippo on df to create vocabulary and fit lds
modelLda, vocab = makeLda(df)

#modelLda.describeTopics().show()

#code from https://datascience.stackexchange.com/questions/20061/words-from-lda-output-pyspark-ml
topics= modelLda.describeTopics()
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()

for idx, topic in enumerate(topics_words):
    print("topic: ", idx)
    print("----------")

    for word in topic:
        print(word)
        print("----------")

[PosixPath('Health-Tweets/gdnhealthcare.txt'), PosixPath('Health-Tweets/NBChealth.txt'), PosixPath('Health-Tweets/usnewshealth.txt'), PosixPath('Health-Tweets/KaiserHealthNews.txt'), PosixPath('Health-Tweets/reuters_health.txt'), PosixPath('Health-Tweets/bbchealth.txt'), PosixPath('Health-Tweets/msnhealthnews.txt'), PosixPath('Health-Tweets/foxnewshealth.txt'), PosixPath('Health-Tweets/cbchealth.txt'), PosixPath('Health-Tweets/cnnhealth.txt'), PosixPath('Health-Tweets/latimeshealth.txt'), PosixPath('Health-Tweets/nytimeshealth.txt'), PosixPath('Health-Tweets/nprhealth.txt'), PosixPath('Health-Tweets/wsjhealth.txt'), PosixPath('Health-Tweets/everydayhealth.txt')]
len(rddarray) 15
len(filenames) 15
+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[0, 1, 2, 4, 5, 3...|[0.00990515200241...|
|    1|[0, 1, 2, 4, 3, 5...|[0.00973377960392...|
|    2|[0, 1, 2, 4, 3, 5...|[0.0096597234240

In [10]:
                
                    #### LOADING PACKAGES ####
    
from pyspark.ml.feature import HashingTF, IDF, StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
    

# we want a function that tests all the different models and return the best one:
def trainbest(df, weights_array):
    # Initialize Multinomial Logistic Regression:
    lr = LogisticRegression(maxIter=5, family="multinomial")
    # define the hashing function:
    hashingTF = HashingTF(inputCol="text", outputCol="features")

    # define the schedule for the ParamGrid search, paramgrid will iterate through different parameters combinations 
    # starting from the hashing function(hashingTF), modifying vector size(numFeatures) and then running the logistic regression(lr) 
    # with different parameters configuration(regParam & elastNetParam).
    schedule = Pipeline(stages=[hashingTF, lr])

    # build up the grid for grid search
    paramGrid = ParamGridBuilder()\
        .addGrid(hashingTF.numFeatures, [5, 8, 6]) \
        .addGrid(lr.regParam, [0.1, 0.03, 0.01]) \
        .addGrid(lr.elasticNetParam, [0.0, 0.3, 1.0])\
        .build()

    # define the evaluator for multiclass classification, this is needed to calculate the accuracy for each iteration and choose the best model, 
    # this is also needed to calculate the accuracy with the test sample.
    MCevaluator = MulticlassClassificationEvaluator(metricName="accuracy")

    # this initialize the grid search, define the schedule, the grid and the evaluator, with the size 
    # for train and validation set fixed to 0.8 of the original train set.    
    tvs = TrainValidationSplit(estimator=schedule,
                               estimatorParamMaps=paramGrid,
                               evaluator=MCevaluator,
                               # 80% of the data will be used for training, 20% for validation.
                               trainRatio=0.8)
    # initialize an empty array to store test accuracy
    test_acc = []
    # initialize an empty array to store models. 
    models = []
    # iterate throught the weight array to split dataset in different size.
    for tr_weight in weights_array:
        # get test size
        te_weight = 1-tr_weight
        # split data with the actual loop stage value
        train, test = df.randomSplit([tr_weight, te_weight])
        # fit model, the model that perform best in the gridsearch with the validation set is memorized
        model = tvs.fit(train)
        # make predictions on test data
        predictions = model.transform(test)
        # calculate accuracy
        accuracy = MCevaluator.evaluate(predictions)
        accuracy = 1.0 - accuracy 
        # store accuray and the model in the arrays for comparison
        test_acc.append(accuracy)
        models.append(model)
        print("... best model for this iteration: Test Error = %g " % (accuracy))
        # repeat...for all the train,test size array...
    # once the loop is over...    
    # check which one has the best accuracy and report the number
    best_acc = max(test_acc)
    #  the best trained model is the one which have obtained the best accuracy for that particular train-test split.
    best_model = models[test_acc.index(max(test_acc))]
    
    return best_acc, best_model

In [11]:
# Initialize RDD #
rdd = initialize(full_rdd)
# Transform to dataframe #
# print(rdd.take(1)) <<<< FOR DEBUG
df = to_indexed_dataframe(rdd)
# define a set of weights to split the dataset #
split_weights = [0.8]
# call the function to train and obtain the best trained model and its accuracy on test data. #
acc, model = trainbest(df, split_weights)

print(" Overall best model has Test Error = %g " % acc)

... best model for this iteration: Test Error = 0.870052 
 Overall best model has Test Error = 0.870052 
